In [ ]:
import sys
import numpy as np
import requests
import shutil
import mediapipe as mp
import cv2
import alphashape
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

# Load data

In [ ]:
def load_dataset(data_path):
    # load file
    # convert it into our dataset
    pass

In [ ]:
data_path = "pose_data"

In [ ]:
data_key = sys.argv[1]
# DB = db_pck.load_db(env_var_db_path)
# data_path = load_path(DB.fetch_path(data_key))
dataset = load_dataset(data_path)

# Fisheye correction

In [ ]:
def LoadCameraMatrix(data_path):
    pass

In [ ]:
def CorrectVideo(cap, camera_matrix, output_path="output/output", show_output = False): #video
    K = camera_matrix.K
    D = camera_matrix.D
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, (width, height), None)
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), new_K, (width, height), cv2.CV_16SC2)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Undistort the frame using the precomputed maps
        undistorted_frame = cv2.remap(frame, map1, map2, interpolation=cv2.INTER_LINEAR)

        if show_output:
        # Show the frame (optional)
            cv2.imshow("Undistorted Video", undistorted_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Write to output file
        
        out.write(undistorted_frame)

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    c_video = cv2.VideoCapture(output_path)
    return c_video
    

In [ ]:
# Mediapose stopgap code
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [ ]:
video = cv2.VideoCapture("test4.mkv")        #dataset.GetVideo()
camera_matrix = LoadCameraMatrix(data_path + "/camera_matrix") # dataset.meta.camera_matrix
c_video = CorrectVideo(video, camera_matrix)

# Pose extraction

In [ ]:
def GetPose(c_video, show_output=False):
    result_list = []
    while c_video.isOpened():
        ret, frame = c_video.read()
        if not ret:
            break

        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Pose
        results = pose.process(rgb_frame)

        # Draw pose landmarks on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
            )
        
        result_list.append(results) #check if deep copy needed

        # Display the frame
        if show_output:
            cv2.imshow('MediaPipe Pose', frame)
            # Exit loop on pressing 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    c_video.release()
    cv2.destroyAllWindows()
    return result_list

In [ ]:
# Mediapipe stopgap code
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(rgb_frame)

    # Draw pose landmarks on the frame
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
        )

    # Display the frame
    cv2.imshow('MediaPipe Pose', frame)

    # Exit loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

In [ ]:
pose_info = GetPose(video) #GetPose(c_video)

# Unit vector extraction

In [ ]:
def ExtractPoints(pose_info):
    pass

In [ ]:
def PointsToUnitVectors(points, camera_frame):
    ones = np.ones((points.shape[0], 1))
    direction_vectors = np.hstack((points.reshape(-1, 2), ones))
    direction_vectors /= np.linalg.norm(direction_vectors, axis=1, keepdims=True)
    return direction_vectors

In [ ]:
def LoadCameraFrame(data_path):
    pass

In [ ]:
points2D = ExtractPoints(pose_info)
camera_frame = LoadCameraFrame(data_path + "/camera_matrix") #dataset.meta.camera_frame
unit_vectors = PointsToUnitVectors(points2D, dataset.meta.camera_frame)

# Conversion to 3D

In [ ]:
def LeastSquares(unit_vectors, point_distance_pairs):
    pass

In [ ]:
def GetConstPairs(data_path):
    pass

In [ ]:
const_pairs = GetConstPairs(data_path + "/rider_keypointpairs") #dataset.meta.rider_info.GetConstPairs()
points3D = LeastSquares(unit_vectors, const_pairs)

# Points to Area

In [ ]:
def Fit3DMesh(points, mesh):
    # Instead of fitting the mesh, just extend an ourline
    pass

In [ ]:
def GetRiderMesh(data_path):
    pass

In [ ]:
mesh = GetRiderMesh(data_path + "/camera_matrix")#dataset.meta.rider_info.GetMesh()
polyline = Fit3DMesh(points3D, mesh)

# Projection to Plane

In [ ]:
def CreatePlane(normal_vec, camera_pos):
    return (normal_vec, camera_pos)

In [ ]:
def ProjectToPlane(polyline, plane):
    polyline[:, 2] = 0  # Set Z-coordinates to 0
    return polyline
    #normal = plane[0] / np.linalg.norm(plane[0])  # Normalize normal vector

    # Compute distances from points to the plane
    #d = np.dot(polyline - plane[1], normal) / np.dot(normal, normal)

    # Compute projected points
    #projected_points = polyline - d[:, np.newaxis] * normal

    #return projected_points

In [ ]:
plane = CreatePlane(dataset.wind_direction)
polyshape = ProjectToPlane(polyline, plane)

# Calculate CdA

In [ ]:
def GetArea(polyshape, alpha = 0.3):
    alpha_shape = alphashape.alphashape(polyshape, alpha)
    if isinstance(alpha_shape, Polygon):  # Ensure it's a polygon, not a multipolygon
        area = alpha_shape.area
        print(f"Alpha Shape Area: {area:.2f} square units")
    else:
        print("Alpha shape resulted in multiple disconnected components")
        return 0 # later iterate through components
    return alpha_shape

In [ ]:
def CalculateCdA(area, coifs=[0.3]):
    return area*coifs[0]

In [ ]:
def CalculateDrag(velocity, cda, air_density=1):
    return 0.5 * air_density * cda * velocity**2

In [ ]:
def CalculateRelative(power, drag, in_per):
    return 100*drag/power

In [ ]:
area = GetArea(polyshape)
CdA = CalculateCdA(area)
drag = CalculateDrag(dataset.velocity, CdA)
rel_power = CalculateRelative(dataset.power, drag)

# Visualize

In [ ]:
#play_video(c_video)
#overlay(pose_info)
#overlay(polyshape)

#draw3D(points3D)
#draw3D(polyline)
#draw2D(polyshape)

#drawLabel(area, CdA, drag, rel_power, dataset.velocity, dataset.power)
#drawGraphs(...)